In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Computer_Science\\Text-Summarization-using-Transformers'

In [4]:
#Entity :

from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path 
    tokenizer_name: Path
    

In [7]:
# configuration 

from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_transformation_config(self):
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name 
        )

        return data_transformation_config




In [11]:
import os 
from src.TextSummarizer.logging import logger 
from transformers import AutoTokenizer 
from datasets import load_dataset, load_from_disk

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length = 128, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return {
            'input_ids' : input_encoding['input_ids'],
            'attention_mask' : input_encoding['attention_mask'],
            'labels' : target_encoding['input_ids']
        }
    

    def convert(self):
        samsum_dataset = load_from_disk(self.config.data_path)
        samsum_dataset_pt = samsum_dataset.map(self.convert_examples_to_featuresm, batched = True)
        samsum_dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [ ]:
#Pipeline : 
try : 
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()


except Exception as e: 
    raise e 